In [1]:
# import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import textdistance
from selenium import webdriver
from selenium.webdriver.common.by import By
from chromedriver_autoinstaller import install
from fuzzywuzzy import process
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re
from validate_email import validate_email

In [2]:
df = pd.read_excel('./../data.xlsx')
df.head()


,company_name,website,address,location,masothue,email,is_email_valid,email_2,is_email_2_valid,phone,...,latitude,longitude,facebook_page,linkedin,masothue_valid,nameCheck,nameRepla,instagram_page,titok_page,facebook_valid
0,Công ty TNHH Ô tô Shineray Việt Nam,shinerayvietnam.com,"Kcn Đại Đồng Hoàn Sơn, xã Hoàn Sơn, Tiên Du, B...",Bắc Ninh,2300321907,Vietnamsrm@gmail.com,True,NaN,NaN,02223847 833,...,21.1056013,105.9881906,https://www.facebook.com/shinerayvietnam.com.vn/,https://www.linkedin.com/pulse/%C4%91%C3%A1nh-...,True,True,NaN,NaN,NaN,True
1,CÔNG TY TNHH DOTTIE,https://dottie.vn/,"285 Cộng Hòa, P13, Q.Tân Bình",Hồ Chí Minh,0315427204,wecare@dottie.vn,True,NaN,NaN,0986216640,...,10.801954,106.644175,https://www.facebook.com/dottiestore/,https://vn.linkedin.com/in/dat-huynh-8230b6187...,True,True,NaN,https://www.instagram.com/dottie.store/?hl=vi,https://www.tiktok.com/@dottie_store,True
2,Công Ty TNHH Eroc Commerce (Việt Nam),https://www.erochome.com/,SGR.01-06.28 Saigon Royal Residence 34 -35 Bến...,Hồ Chí Minh,0316242284,contact@careerbuilder.vn.Tra,False,NaN,NaN,02822508610,...,10.7687405,106.7017541,https://www.facebook.com/erocvietnam,https://vn.linkedin.com/in/thom-phan-a7119b256,True,True,NaN,NaN,NaN,True
3,Công Ty TNHH Mediastep Software Việt Nam,http://www.mediastep.com/,"Lầu 11, Vĩnh Trung Plaza , 255 Hùng Vương, Quậ...",Đà Nẵng,0314449727,hotro@gosell.vn,True,info@mediastep.com,True,NaN,...,16.0668084,108.2131037,https://www.facebook.com/mediastepsoftware/?lo...,https://vn.linkedin.com/company/mediastep-soft...,True,True,NaN,NaN,NaN,True
4,CÔNG TY CỔ PHẦN DƯỢC MỸ PHẨM A.Q.P VIỆT NAM,https://aqpgroup.vn/,"Số 6 ngõ 18 đường Võ Văn Dũng, Phường Ô Chợ Dừ...",Hà Nội,0108899696,tuyendung@topcv.vn,True,NaN,NaN,NaN,...,21.0154985,105.8232701,https://m.facebook.com/people/C%C3%B4ng-Ty-C%E...,https://www.linkedin.com/in/aqpgroup/,True,True,NaN,https://www.instagram.com/aqpgroup/,NaN,True


## Kiểm tra độ chính xác của cột Mã số thuế và Company_name

In [3]:
# khổi tạo chorme driver 
install()

'c:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python38\\lib\\site-packages\\chromedriver_autoinstaller\\113\\chromedriver.exe'

In [4]:
def check_mst(name,mst):
    driver = webdriver.Chrome()
    driver.get('https://dichvuthongtin.dkkd.gov.vn/inf/Forms/Products/ProductCatalog.aspx?h')
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR,'.ProdGroupPage #FrontPG-2').click()
    driver.find_element(By.CSS_SELECTOR,'.ProdGroupPage .FrontProdGroups #FrontPGT-2 #LnkActiveProdGroup').click()
    driver.find_element(By.CSS_SELECTOR,'.columns .Filter #ctl00_C_UC_ENT_LIST1_ENTERPRISE_GDT_CODEFilterFld').send_keys(mst)
    time.sleep(0.5)
    driver.find_element(By.CSS_SELECTOR,'.wwDivFilter input[value*="Tìm kiếm"]').click()
    time.sleep(2)
    text_mst =  driver.find_elements(By.CSS_SELECTOR,'.gridview tbody tr a')[2].text
    driver.quit()
    
    check = textdistance.ratcliff_obershelp(name.upper(), text_mst)
    if check > 0.75:
        check = 'True'
    else:
        check = 'False'
        
    return check

In [ ]:
a= df[['company_name','masothue']].agg(list, axis=1)
count = 0
driver = webdriver.Chrome()
driver.get('https://dichvuthongtin.dkkd.gov.vn/inf/Forms/Products/ProductCatalog.aspx?h')
time.sleep(1)
driver.find_element(By.CSS_SELECTOR,'.ProdGroupPage #FrontPG-2').click()
driver.find_element(By.CSS_SELECTOR,'.ProdGroupPage .FrontProdGroups #FrontPGT-2 #LnkActiveProdGroup').click()
for i,j in a : 
    driver.find_element(By.CSS_SELECTOR,'.columns .Filter #ctl00_C_UC_ENT_LIST1_ENTERPRISE_GDT_CODEFilterFld').send_keys(j)
    time.sleep(0.5)
    driver.find_element(By.CSS_SELECTOR,'.wwDivFilter input[value*="Tìm kiếm"]').click()
    time.sleep(2)
    if driver.find_element(By.CSS_SELECTOR,'i').text.split(': ')[1] != '0':
        df['masothue_valid'][count] = 'True'
        
        text_mst =  driver.find_elements(By.CSS_SELECTOR,'.gridview tbody tr a')[2].text

        check = textdistance.ratcliff_obershelp(i.upper(), text_mst)
        if check > 0.75:
            df['nameCheck'][count] = 'True'
            df['nameRepla'][count] = ''
        else:
            df['nameCheck'][count] = 'False'
            df['nameRepla'][count] = text_mst
    else:
        df['masothue_valid'][count] = 'False'
    driver.find_element(By.CSS_SELECTOR,'.columns .Filter #ctl00_C_UC_ENT_LIST1_ENTERPRISE_GDT_CODEFilterFld').clear()
    count+=1
driver.quit()


## Lấy thêm dữ liệu từ website công ty

In [19]:
driver = webdriver.Chrome()
driver.get('https://dottie.vn/')

In [ ]:
link = set()
a = driver.find_elements(By.TAG_NAME,'a')
for i in a:
    try:
        if 'facebook' in i.get_attribute('href'):
            link.add(i.get_attribute('href'))
        if 'instagram' in i.get_attribute('href'):
            link.add(i.get_attribute('href'))
        if 'tiktok' in i.get_attribute('href'):
            link.add(i.get_attribute('href'))
        if 'linkedin' in i.get_attribute('href'):
            link.add(i.get_attribute('href'))
    except:
        i = i
print(link)


In [52]:
count = 0
df['email_2']=df['email_2'].astype(str)
df['is_email_2_valid']=df['is_email_2_valid'].astype(str)
name_phone = df[['website', 'phone']].agg(list,axis=1)
for i,j in name_phone :
    
    if not i.startswith('http'):
        i= 'https://' + i       
    driver = webdriver.Chrome()
    try:
        driver.get(str(i))
    except:
        print(i)
    a = driver.find_elements(By.TAG_NAME,'a')

    for href in a:
        try:
            if 'facebook' in href.get_attribute('href'):
                df.at[count,'facebook_page']= href.get_attribute('href')
            if 'instagram' in href.get_attribute('href'):
                df.at[count,'instagram_page']= href.get_attribute('href')
            if 'tiktok' in href.get_attribute('href'):
                df.at[count, 'titok_page']= href.get_attribute('href')
            if 'linkedin' in href.get_attribute('href'):
                df.at[count, 'linkedin']= href.get_attribute('href')
        except:
            href    
    span = driver.find_elements(By.TAG_NAME,'span')
    
    for span_text in span:
        try:
            emails = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', span_text.text)
            phone_numbers = re.findall(r'\b\d{10}\b', span_text.text)
            if len(emails) != 0 :
                for email in emails:
                    if df['email'][count] != email:
                        df.at[count,'email_2'] = email
                        is_valid = validate_email(email, check_mx=True)

                        if is_valid:
                            df.at[count,'is_email_2_valid'] = 'True'
                        else:
                            df.at[count,'is_email_2_valid'] = 'Flase'
            if len(phone_numbers) !=0:
                for phone_number in phone_numbers:
                    
                    if df['phone'][count] != phone_number:
                        df.at[count,'phone_2'] = str(phone_number)
        except:
            i
    count+=1
    driver.quit()

http://sunrisecharcoal.vn
https://www.olalani.com
https://sailfish.vn
https://vinamedical.vn
https://goldenctity.com
https://vietnamoto.com.vn
https://www.trivin-sa.com
https://sangtamgroup.com
https://pefoam.bizz.vn


## Kiểm tra facebook_page

In [65]:
def count_matching_chars(str1, str2):
    count = 0
    for i in range(len(str1)):
        if i < len(str2) and str1[i] == str2[i]:
            count += 1
    return count

In [91]:
df['facebook_page'] = df['facebook_page'].fillna('')

In [94]:
count = 0
for i in df['facebook_page']:
    if i == '':
        count+=1
        continue
    driver = webdriver.Chrome()
    driver.get(i)

    # print(soup.find('h1').text)
    title = driver.find_element(By.TAG_NAME,'title')
    tmp = title.get_attribute('text').split('-')[0].strip().split('|')[0].strip()
    check = textdistance.ratcliff_obershelp(tmp, df['company_name'][count])
    if tmp.split(' ')[0].upper() in df['company_name'][count].upper() or check > 0.5 :
        df.at[count, 'facebook_valid'] = 'True'
    else:
        df.at[count, 'facebook_valid'] = 'False'
    driver.quit()
    count+=1

0.7076923076923077
0.08
0.40816326530612246
0.64
0.5348837209302325
0.0
0.14035087719298245
0.19672131147540983
0.12121212121212122
0.0425531914893617
0.6666666666666666
0.7096774193548387
0.15625
0.896551724137931
0.5205479452054794
0.2545454545454545
0.08163265306122448
0.25
0.6857142857142857
0.7948717948717948
0.17142857142857143
0.24489795918367346
0.26666666666666666
1
0.34615384615384615
0.4
0.0851063829787234
0.5185185185185185
0.15384615384615385
1
0.7567567567567568
0.8285714285714286
0.15
0.0425531914893617
0.10810810810810811
0.14705882352941177
0.20833333333333334
0.29411764705882354
0.5454545454545454
0.47619047619047616
0.5862068965517241
0.11594202898550725
0.8571428571428571
0.11764705882352941
0.0
0.5806451612903226
0.8615384615384616
0.13793103448275862
0.11764705882352941
0.37777777777777777
0.4
0.14634146341463414
0.0
0.4230769230769231
0.5294117647058824
0.11764705882352941
0.41379310344827586
0.16666666666666666
0.2903225806451613
0.3888888888888889
0.77777777777

## SEARCH LINK_FB bằng Facebook Search Engine

In [5]:
def fb_Search(companyname):
    count = 0
    data = pd.DataFrame()
    driver = webdriver.Chrome()
    driver.get('https://www.social-searcher.com/facebook-search')
    for i in companyname:
        name = i
        if 'TNHH' in i.upper(): 
            name = (i.upper().replace('CÔNG TY TNHH','') )
        elif 'CP' in i.upper():
            name = (i.upper().replace('CÔNG TY CP','') )
        else:
            name = (i.upper().replace('CÔNG TY CỔ PHẦN','') )
        driver.find_element(By.CSS_SELECTOR,'#facebooksearchinput').send_keys(name)
        driver.find_element(By.CSS_SELECTOR,'.facebook-page-form__btn').click()
        time.sleep(1)
        facebook_link = driver.find_element(By.CSS_SELECTOR,'.gsc-wrapper a').get_attribute('href')
        print(facebook_link)
        driver.find_element(By.CSS_SELECTOR,'#facebooksearchinput').clear()
        time.sleep(2)
        data.at[count, 'link_fb'] = 'https://www.facebook.com/' + facebook_link.split('/')[3] + '/about'
        count+=1
    driver.quit()
    data.to_excel('./../linkfb.xlsx', index=False)

## Crawls facebook.com

In [6]:
def findFather(child, nameFather):
    
    parent_element = child
    while parent_element.tag_name != nameFather:
        parent_element = parent_element.find_element(By.XPATH,'..')
    return parent_element.text
def next_(present):
    present = present.find_element(By.XPATH,"following-sibling::*[1]")
    return present
def parent(present):
    present = present.find_element(By.XPATH,'..')
    return present

In [7]:
def Type2(driver,result):
    Domain = driver.current_url.split('/')[-2]
    name               = driver.title.split('|')[0].split('-')[0].strip()
    emailValidate = ''
    try:
        logo                = driver.find_element(By.CSS_SELECTOR, f'svg[aria-label="{name}"]').find_element(By.CSS_SELECTOR,'g image').get_attribute('xurl:href')
    except: logo            = ''
    try:
        email     = findFather(driver.find_element(By.XPATH, "//div[text()='Email']"),'li').split('\n')[0]
    except: email     = ''
    if email != '':
        is_valid = validate_email(email, check_mx=True)
        if is_valid:
            emailValidate = 'True'
        else:
            emailValidate = 'False'    
    try:
        telephone     = findFather(driver.find_element(By.XPATH, "//div[text()='Mobile']"),'li').split('\n')[0]
    except: telephone     = ''
    try:
        website   = findFather(driver.find_element(By.XPATH, "//div[text()='Website']"),'li').split('\n')[0]
    except: website     = ''
    try:
        child_Hours     = findFather(driver.find_element(By.XPATH, "//div[text()='Hours']"),'li').split('\n')[0]
    except: child_Hours     = ''
    try:
        categories      = next_(parent(driver.find_element(By.CSS_SELECTOR,'img[src="https://static.xx.fbcdn.net/rsrc.php/v3/yr/r/lhdCVH10kLz.png"]'))).text.split('\n')[0]
    except: categories      = ''
    try:
        address         = next_(parent(driver.find_element(By.CSS_SELECTOR,'img[src="https://static.xx.fbcdn.net/rsrc.php/v3/yS/r/poZ_P5BwYaV.png"]'))).text.split('\n')[0]
    except: address         = ''
    try:
        ratingCheckin            = next_(parent(driver.find_element(By.CSS_SELECTOR,'img[src="https://static.xx.fbcdn.net/rsrc.php/v3/yz/r/IawsY0Yjegk.png"]'))).text.split('\n')[0]
    except: ratingCheckin            = ''
    try:
        rantingFollow          = driver.find_element(By.CSS_SELECTOR, f'a[href="https://www.facebook.com/{Domain}/followers/"]').text.split(' ')[0]
    except: rantingFollow          = ''
    try:
        ratingLike            = driver.find_element(By.CSS_SELECTOR, f'a[href="https://www.facebook.com/{Domain}/friends_likes/"]').text.split(' ')[0] 
    except: ratingLike            = ''
    latitude = ''
    longitude = ''
    try:
        Coordinates = parent(parent(driver.find_element(By.CSS_SELECTOR, 'div[aria-label="View map info"]'))).find_element(By.CSS_SELECTOR,"div").get_attribute('style')

        match = re.search(r'center=([\d\.]+)%2C([\d\.]+)', Coordinates)
        
        if match:
            latitude = match.group(1)
            longitude = match.group(2)
            
    except: Coordinates         = ''
    # crawl 
    driver.execute_script("window.scrollTo(0, 200);")
    driver.find_element(By.XPATH, "//span[text()='Page transparency']").click()
    time.sleep(1)
    try:
        pageID         = next_(parent(driver.find_element(By.CSS_SELECTOR,'img[src="https://static.xx.fbcdn.net/rsrc.php/v3/y-/r/olD6qzqyixJ.png"]'))).text.split('\n')[0]
    except: pageID         = ''
    try:
        dateCreated         = next_(parent(driver.find_element(By.CSS_SELECTOR,'img[src="https://static.xx.fbcdn.net/rsrc.php/v3/yG/r/p7Pf4gSWotr.png"]'))).text.split('\n')[0]
        date_str            = dateCreated
        date_obj            = datetime.strptime(date_str, '%d %B %Y')
        dateCreated         = date_obj.strftime('%d/%m/%Y')
    except: dateCreated         = ''
    try:
        owns         = next_(parent(driver.find_element(By.CSS_SELECTOR,'img[src="https://static.xx.fbcdn.net/rsrc.php/v3/yV/r/VHDIl6nNAEW.png"]'))).text.split('\n')[1]
    except: owns         = ''
    try:
        advertisement         = next_(parent(driver.find_element(By.CSS_SELECTOR,'img[src="https://static.xx.fbcdn.net/rsrc.php/v3/yb/r/Ix9L9CJxYBf.png"]'))).text
    except: advertisement         = ''
    
    url                    = driver.current_url.split('/about')[0]
    
    result                      = {
        'name'          : name,
        'categories'    : categories,
        'logo'          : logo,
        'email'         : email,
        'emailValidate' : emailValidate,
        'telephone'     : telephone,
        'website'       : website,
        'HoursOpen'     : child_Hours,
        'pageID'        : pageID,
        'dateCreated'   : dateCreated,
        'owns'          : owns,
        
        }   
     
    result['place']             = {
        'address'       : address,
        'latitude'      : latitude,
        'longitude'     : longitude
    }
    
    result['ratingValue']       = {
        'ratingCheckin' : ratingCheckin,
        'ratingLike'    : ratingLike,
        'rantingFollow' : rantingFollow,
    }
    result['advertisement']     = advertisement
    result['sourceBrandname']   =  'Facebook'
    result['url'] =  url
    
    return result


In [8]:
def Type1(driver,result):
    name               = driver.title.split('|')[0].split('-')[0].strip()
    logo                = driver.find_element(By.CSS_SELECTOR,'g[mask] image').get_attribute('xlink:href')
    latitude = ''
    longitude = ''
    Info = parent(parent(driver.find_element(By.XPATH, "//span[text()='ADDITIONAL CONTACT INFO']"))).text.replace(' ', '')
    try:
        Coordinates         = parent(parent(driver.find_element(By.CSS_SELECTOR, 'div[aria-label="View map info"]'))).find_element(By.CSS_SELECTOR,"div").get_attribute('style')

        match = re.search(r'center=([\d\.]+)%2C([\d\.]+)', Coordinates)
        
        if match:
            latitude = match.group(1)
            longitude = match.group(2)
    except: Coordinates         = ''
    try:
        address             = next_(parent(driver.find_element(By.CSS_SELECTOR, 'div[aria-label="Map view"]'))).text
    except : address = ''
    try:
        ratingLike             = parent(driver.find_element(By.XPATH, "//span[text()='like this']")).text.split(' ')[0]
    except : ratingLike = ''
    try:
        ratingFollow             = driver.find_element(By.XPATH, "//span[contains(text(), 'follow this') or contains(text(), 'follow this')]").text.split(' ')[0]
    except : ratingFollow = ''
    try:
        ratingCheckin             = parent(driver.find_element(By.XPATH, "//span[text()=' checked in here']")).text.split(' ')[0]
    except : ratingCheckin = ''
    try:
        categories             = parent(parent(parent(parent(driver.find_element(By.XPATH, "//span[text()='GENERAL']"))).find_element(By.CSS_SELECTOR,'a'))).text
    except : categories = ''
    try:
        website             = re.findall(r"(?P<web>http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)", Info)[0]
    except :website = ''
    try:
        telephone             = re.findall(r'\b\d{10}\b', Info)[0]
    except : telephone = ''
    try:
        email             = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', Info)[0]
    except : email = ''
    try:
        openingHours             = parent(parent(driver.find_element(By.XPATH,'//i[contains(@style, "background-image") and contains(@style, "-147px -145px")]'))).text
    except : openingHours = ''
    try:
        abstract             = parent(parent(parent(driver.find_element(By.XPATH, "//span[text()='MORE INFO']")))).text.replace('About\n','').replace('Additional information','').split('\n')[1:]
    except : abstract = ''
    
    
    url                    = driver.current_url.split('/about')[0]
    
    result     = {
        'name'          : name,
        'categories'    : categories,
        'logo'          : logo,
        'email'         : email,
        'telephone'     : telephone,
        'website'       : website,
        'openingHours'  : openingHours,
        }    
    result['place'] = {
        'address'       : address,
        'latitude'      : latitude,
        'longitude'     : longitude
        }
    result['ratingValue'] = {
        'ratingCheckin'       : ratingCheckin,
        'ratingLike'          : ratingLike,
        'ratingFollow'        : ratingFollow
    }
    result['abstract']      = abstract
    result['sourceBrandname'] =  'Facebook'
    result['url'] =  url
    
    return result

In [9]:
link_fb = pd.read_excel('./../linkfb.xlsx')

In [10]:
link_fb['link_fb'][0]

'https://www.facebook.com/shinerayvietnam.com.vn/about'

In [11]:
def run(link):
    result = {}
    driver = webdriver.Chrome()
    driver.get(link)
    try:
        try:
            if driver.find_element(By.CSS_SELECTOR,'h2').text == "This page isn't available":
                driver.quit()
                return result
            time.sleep(2)
            driver.execute_script("window.scrollTo(0, 200);")
            driver.find_element(By.XPATH, "//span[text()='Page transparency']")
            result = Type2(driver, result)
            driver.quit()
            return result
        except:     
            time.sleep(2)
            result = Type1(driver, result)
            driver.quit()
            return result
    except:
        driver.quit()
        return result

In [13]:
test = []
test.append(run('https://www.facebook.com/shinerayvietnam.com.vn/about'))
test

[{'name': 'Nhà Máy Ô Tô Shineray Việt Nam',
  'categories': 'Cars · Catering van · New vehicles',
  'logo': 'https://scontent.fsgn5-13.fna.fbcdn.net/v/t39.30808-1/327126861_1320963885304342_3219006542954949654_n.png?stp=dst-png_p148x148&_nc_cat=106&ccb=1-7&_nc_sid=1eb0c7&_nc_ohc=psJ2uUgDwiQAX9F-RU8&_nc_ht=scontent.fsgn5-13.fna&oh=00_AfDJb1M3blRq4ps5pG9_ysBDUQwEzkV2VG75m9tCV4ai4g&oe=646B49FE',
  'email': 'vietnamsrm@gmail.com',
  'telephone': '',
  'website': 'http://shineraymotor.vn/',
  'openingHours': '',
  'place': {'address': 'Khu công nghiệp Đại Đồng- Hoàn Sơn- Tiên Du- Bắc Ninh Bac Ninh, Bắc Ninh Province, Vietnam',
   'latitude': '21.100355948592',
   'longitude': '105.97525726318'},
  'ratingValue': {'ratingCheckin': '24',
   'ratingLike': '10,514',
   'ratingFollow': '10,658'},
  'abstract': ['Xe Tải Nhẹ SRM , DongBen',
   'Website: https://www.shineraymotor.vn/',
   'Hotline: 0886 893 355',
   'Địa chỉ: Khu công nghiệp Đại Đồng- Hoàn Sơn- Tiên Du- Bắc Ninh'],
  'sourceBrandna

In [130]:
str(test).replace('[','{').replace(']','')

"{{'name': 'Nhà Máy Ô Tô Shineray Việt Nam', 'categories': 'Cars · Catering van · New vehicles', 'logo': 'https://scontent.fsgn5-13.fna.fbcdn.net/v/t39.30808-1/327126861_1320963885304342_3219006542954949654_n.png?stp=dst-png_p148x148&_nc_cat=106&ccb=1-7&_nc_sid=1eb0c7&_nc_ohc=b-UKdF6IG7cAX9ies-2&_nc_ht=scontent.fsgn5-13.fna&oh=00_AfClWMxaggVd4l26GK3o2laoguyNNWIaMhfxs9_FbWqvGw&oe=6467557E', 'email': 'vietnamsrm@gmail.com', 'telephone': '', 'website': 'http://shineraymotor.vn/', 'openingHours': 'Always open', 'place': {'address': 'Khu công nghiệp Đại Đồng- Hoàn Sơn- Tiên Du- Bắc Ninh Bac Ninh, Bắc Ninh Province, Vietnam', 'latitude': '21.100355948592', 'longitude': '105.97525726318'}, 'ratingValue': {'ratingCheckin': '24', 'ratingLike': '10,515', 'ratingFollow': '10,658'}, 'abstract': {'Xe Tải Nhẹ SRM , DongBen', 'Website: https://www.shineraymotor.vn/', 'Hotline: 0886 893 355', 'Địa chỉ: Khu công nghiệp Đại Đồng- Hoàn Sơn- Tiên Du- Bắc Ninh'], 'sourceBrandname': 'Facebook', 'url': 'http

In [136]:
len(link_fb['link_fb'])

99

In [123]:
import json

with open("./../result1.json", "w") as outfile:
    json.dump(test, outfile)


In [132]:
with open("./../result1.json", "w", encoding="utf-8") as f:
    json.dump(test, f, ensure_ascii=False, indent=4)  


In [54]:
df.to_excel('./../data.xlsx', index=False)